#　Python Control 練習
次のセルを実行して、python-controlをインストールします。

セルの実行は左の▷を押すか、セルを選択してshift+enterです・

In [ ]:
!pip install control

In [ ]:
#必要なモジュールを読み込む
import control as ctrl
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#伝達関数オブジェクトを作成
sys=ctrl.tf( [110],  [0.01, 1 , 0] )
print(sys)


In [ ]:
#システムの安定余裕を調べる
gm, pm, gcw, pcw = ctrl.margin(sys)
print("ゲイン余裕:", gm)
print("位相余裕:", pm)
print("位相交差周波数:", gcw)
print("ゲイン交差周波数:", pcw)


In [ ]:
#ステップ応答を計算
t, y = ctrl.step_response(sys)
plt.figure()
plt.plot(t, y)
plt.title("Step Response")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.grid()
plt.show()


In [ ]:
#ボード線図を描く

ctrl.bode(sys)

In [ ]:
#複数のボード線図を重ねる
sys1 = ctrl.tf([1], [1, 3, 2])
sys2 = ctrl.tf([5], [1, 1])
ctrl.bode([sys1, sys2])

In [ ]:
# 複数のシステムの直列接続
newsys = sys1*sys2
ctrl.bode(newsys)

# ループ成形チャレンジ

## StampFlyの伝達関数を生成

In [ ]:
#必要なモジュールを読み込む
import control as ctrl
import numpy as np
import matplotlib.pyplot as plt

roll_gain= 219.4161359941975
pitch_gain= 151.11667712081575
yaw_gain= 29.43538617407769
time_constant_stampfly = 0.011727677322467796

#Roll, Pitch, Yawの伝達関数を生成
sys_roll = ctrl.tf( [roll_gain],  [time_constant_stampfly, 1 , 0] )
sys_pitch = ctrl.tf( [pitch_gain],  [time_constant_stampfly, 1 , 0] )
sys_yaw = ctrl.tf( [yaw_gain],  [time_constant_stampfly, 1 , 0] )

#表示
print("Rollの伝達関数")
print(sys_roll)
print("Pitchの伝達関数")
print(sys_pitch)
print("Yawの伝達関数")
print(sys_yaw)

In [ ]:
# Bode線図を描く

ctrl.bode([sys_roll, sys_pitch, sys_yaw])
plt.legend(["Roll", "Pitch", "Yaw"])
plt.show()
